### SUMMARY
**0) Preliminary works**
> 0.1) Prepare sample file <br>
> 0.2) read_csv with nrows option<br>

**1) Get familiar with data**<br>
**2) Select columns of interest**<br>
**3) What to do with NaN**<br>
**4) Make an action plan**<br>
**5) Adjust the code to work with Big data**<br>
> 5.1) Create an ITERATOR from the data base and define CHUNK size <br>
> 5.2) Test Big Data approach on a sample

In [ ]:
#GENERAL SETTINGS
import pandas as pd
pd.set_option('display.max_columns', None) #so we can see all the file columns

#or
pd.options.display.max_columns = None

## 0) Preliminary works

If we don´t want to read the whole file

#### 0.1) Prepare sample file
with command line

In [ ]:
#TAKING FIRST 10000 ROWS AND CREATING A SAMPLE FILE
!bzcat bookings.csv.bz2 | head -10000 > bookings.sample.csv
#READ FILE WITH PANDAS
b = pd.read_csv('bookings.sample.csv.bz2', sep='^')

with Pandas

In [ ]:
#TAKING FIRST 10000 ROWS
b = pd.read_csv('bookings.csv.bz2', sep='^', nrows=9999)

#TAKING RANDOM SAMPLE OF 10000 ROWS
import random

p = 0.01
b_muestra = pd.read_csv('bookings.csv.bz2',sep='^',header=0,
                        skiprows=lambda i: i>0 and random.random() > p)

#### 0.2) read_csv with nrows option

In [ ]:
#TAKE JUST 10000 LINES
b = pd.read_csv('bookings.csv.bz2', sep='^', nrows=9999)

## 1) Get familiar with data

In [ ]:
#USE THE FOLLOWING FUNCTIONS
b.shape         #check nº of rows and columns
list(b.columns) #check white spaces in column names
b.sample()      #check random sample
                #ask for the data dictionary and study all fields
b.info()        #check data types and spot nulls
b.describe()    #check descriptive statistics

#Analyze date with:
groupby()
count()
sum()
sort_values()
head()

## 2) Select columns of interest

In [ ]:
b = pd.read_csv('bookings.csv.bz2', sep='^', usecols=['arr_port', 'pax', 'year'], nrows=9999)

## 3) What to do with NaN

In [ ]:
#decide whether to eliminate columns/rows with NaNs
#or replace NaNs with other values
b = b.dropna()

## 4) Make an action plan

Elaborate a list of actions to process your code.
One action can be equal to one command to execute.

EXAMPLE:

1) filter 2013 rows
2) groupby arr_port
3) sum pax
4) sort by pax

Once you have the code, put it together in a single cell so you can execute it at once without worrings in which cells that or that variable was created, etc...
Example below:

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
b = pd.read_csv('bookings.csv.bz2',sep='^', usecols=['arr_port', 'pax', 'year'], nrows=9999)
b = b.dropna()
b=b[ b['year']==2013 ]
del b['year']
b.groupby(['arr_port']).sum().sort_values('pax', ascending=False).head()

## 5) Adjust the code to work with Big data

Once we now that our code works with the sample file, we need to run it in chunks over the original data base.

### 5.1) Create an ITERATOR from the data base and define CHUNK size

Our machine does not normally has enough memory to read an entire big CSV into a DataFrame. The solution is to process the CSV in CHUNKS.

To read a file in chunks we need an ITERATOR OBJECT.
When you read a file in chunks, the cursor moves through the rows. If the last chunks exceeds the last row, you will get an error.

An ITERATOR OBJECT has the property of stopping the reading process without causing an error.

In [ ]:
#this code returns an iterator instead of a DataFrame
bi = pd.read_csv('bookings.csv.bz2', sep='^', usecols=['arr_port', 'pax', 'year'], nrows=9999, iterator=True)

#this code returns a chunk of the file
bi.get_chunk()

In [ ]:
#define CHUNK SIZE with read_csv also returns and ITERATOR with a predefined chunk size
bi = pd.read_csv('bookings.csv.bz2', sep='^', usecols=['arr_port', 'pax', 'year'], nrows=9999, chunksize=3000)

### 5.2) Test Big Data approach on a sample

The Big Data approach is to run the code by file chunks and append the results at the end.

We have to test this approach in smaller chunks, taken from the sample of 10000 rows.

In [ ]:
#RECAP 1
#this is the code we were running on the above sample
b = b.dropna()
b=b[ b['year']==2013 ]
del b['year']
b.groupby(['arr_port']).sum()

In [ ]:
#RECAP 2
#set options
import pandas as pd
pd.set_option('display.max_columns', None)

#get 10000 rows sample iterator with 3000 chunks
bi = pd.read_csv('bookings.csv.bz2', sep='^', usecols=['arr_port', 'pax', 'year'], nrows=9999, chunksize=3000)

#declare empty DataFrame to store results of each chunk
all_chunks=pd.DataFrame()

#iterate through chunks and append results
for i, b in enumerate(bi):
    print(i, len(b))    
    b=b.dropna()
    b=b[b['year']==2013]
    del b['year']
    result=b.groupby(['arr_port']).sum()
    
    all_chunks=all_chunks.append(result)

#groupby puts the grouping column as index so we have to:
#reset_index()
#groupby again the all_chunks results > sum > sort > reset_index again
result_all = all_chunks.reset_index().groupby(['arr_port']).sum().sort_values(by='pax', ascending=False).reset_index()
result_all.head(10)


Check if the result_all is the same as the one executed for a single sample (without chunks)

## 6) Run program with Big Data

In [ ]:
#Meaning, don´t take just 9999 rows but all (removing 'nrows=9999')
bi = pd.read_csv('bookings.csv.bz2', sep='^', usecols=['arr_port', 'pax', 'year'], chunksize=3000)

Get familiar with data
Select the columns of interest
What to do with Na?

Make action plan
Develop the code that works with a sample

5 Adjust the code to work with Big data
Test big data approach on a sample

6 Run program with big data